In [4]:
class BinTreeNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        
    @property
    def value(self):
        return self.__value
    
    @value.setter
    def value(self, value):
        self.__value = value
        
    @property
    def left(self):
        return self.__left
    
    @left.setter
    def left(self, left):
        self.__left = left
    
    @property
    def right(self):
        return self.__right
    
    @right.setter
    def right(self, right):
        self.__right = right

In [107]:
class BinTree:
    def __init__(self):
        self.root = None
    
    def __iterate_tree(self, root):
        if root == None:
            return
        
        yield from self.__iterate_tree(root.left)
        yield root.value
        yield from self.__iterate_tree(root.right)
    
    def __search_value(self, root, parent, value):
        if root == None:
            return None, parent
        
        if value == root.value:
            return root, parent
        
        if value < root.value:
            return self.__search_value(root.left, root, value)
        
        return self.__search_value(root.right, root, value)
    
    def __iter__(self):
        return self.__iterate_tree(self.root)
    
    def __len__(self):
        return sum(1 for _ in iter(self))
    
    def __add__(self, value):
        self.insert(value)
    
    def insert(self, value):
        node = BinTreeNode(value)
        
        if self.root == None:
            self.root = node
            return
        
        _, parent = self.__search_value(self.root, None, value)
        
        if value < parent.value:
            parent.left = node
        else:
            parent.right = node
    
    def search(self, value):
        node, _ = self.__search_value(self.root, None, value)
        return node
    
    def contains(self, value):
        return self.search(value) != None

In [108]:
t = BinTree()
t.insert(10)
t.insert(9)
t.insert(11)
t + 12
t + 5
len(t)

5

In [109]:
it = iter(t)
list(it)

[5, 9, 10, 11, 12]

In [110]:
t.contains(20)

False